In [1]:
import discord
import nest_asyncio
import os
import logging
from discord.ext import commands
from discord.ui import Button, View, Modal, TextInput
from flask import Flask
from threading import Thread

# Set up logging
logging.basicConfig(level=logging.INFO, filename='bot.log', filemode='w', format='%(asctime)s - %(levelname)s - %(message)s')

app = Flask('')

@app.route('/')
def home():
    return "Hello. I am alive!"

def run_flask():
    app.run(host='0.0.0.0', port=8080)

def keep_alive():
    t = Thread(target=run_flask)
    t.start()

nest_asyncio.apply()

intents = discord.Intents.default()
intents.message_content = True  # Ensure this is set to True to read message content
bot = commands.Bot(command_prefix='!', intents=intents)

@bot.event
async def on_ready():
    logging.info(f'Bot is ready. Logged in as {bot.user}')

# Custom Help Command
@bot.command(name="sagihelp")
async def sagihelp(ctx):
    await ctx.send("""
    **Available Commands:**
    - `!sagihelp`: Shows this help message.
    - `!verify`: Initiates the verification process.
    - `!status`: Displays the bot's current status.
    """)

# Status Command
@bot.command()
async def status(ctx):
    await ctx.send(f"""
    **Bot Status:**
    - **Online**
    - **Latency:** {round(bot.latency * 1000)}ms
    """)

# Verify Command
@bot.command()
async def verify(ctx):
    button = Button(label="Get Verified", style=discord.ButtonStyle.green)

    async def button_callback(interaction):
        modal = Modal(title="Verification")

        nickname = TextInput(label="In-Game Nickname", placeholder="What is your in-game nickname? Ex: Sagi")
        server = TextInput(label="Current Server", placeholder="What is your current server ? Ex: 3322", max_length=4)

        modal.add_item(nickname)
        modal.add_item(server)

        async def modal_callback(interaction):
            try:
                if not server.value.isdigit() or not (1 <= int(server.value) <= 9999):
                    await interaction.response.send_message("Server must be a number between 1 and 9999.", ephemeral=True)
                    return

                new_nickname = f"{server.value} | {nickname.value}"

                await interaction.user.edit(nick=new_nickname)

                guild = interaction.guild
                verified_role = discord.utils.get(guild.roles, name="Verified")
                not_verified_role = discord.utils.get(guild.roles, name="Not Verified")

                if verified_role and not_verified_role:
                    await interaction.user.add_roles(verified_role)
                    await interaction.user.remove_roles(not_verified_role)
                    await interaction.response.send_message(f"Nickname updated to {new_nickname} and roles updated.", ephemeral=True)
                else:
                    await interaction.response.send_message("Verified or Not Verified role not found.", ephemeral=True)

            except discord.Forbidden:
                await interaction.response.send_message("I don't have permission to change your nickname.", ephemeral=True)
                logging.error(f"Forbidden error: Unable to change nickname for {interaction.user}.")
            except discord.HTTPException as e:
                await interaction.response.send_message("An error occurred while processing your request. Please try again later.")
                logging.error(f"HTTPException: {e} while processing verification for {interaction.user}.")
            except Exception as e:
                await interaction.response.send_message("An unexpected error occurred. Please try again later.")
                logging.error(f"Unexpected error: {e} during verification for {interaction.user}.")

        modal.on_submit = modal_callback
        await interaction.response.send_modal(modal)

    button.callback = button_callback

    view = View()
    view.add_item(button)

    await ctx.send("Press the button to get verified:", view=view)

# Keep the bot alive with the web server
keep_alive()

# Run the bot
bot.run('TOKEN')

[2024-06-21 14:15:31] [INFO    ] discord.client: logging in using static token


 * Serving Flask app ''
 * Debug mode: off


[2024-06-21 14:15:31] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: d686881b6ce204342e8b1c4404040373).
